# CS301 Project

Source: https://www.kaggle.com/datasets/parisrohan/credit-score-classification

## Preprocessing

In [28]:
# import packages
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder, RobustScaler, StandardScaler, Normalizer
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix

In [29]:
# read in data
data = pd.read_csv('./data/train.csv', low_memory=False)

data.drop(['ID', 'Customer_ID', 'Month', 'Name', 'SSN'], axis=1, inplace=True)

pd.set_option('display.max_columns', None)
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 23 columns):
 #   Column                    Non-Null Count   Dtype  
---  ------                    --------------   -----  
 0   Age                       100000 non-null  object 
 1   Occupation                100000 non-null  object 
 2   Annual_Income             100000 non-null  object 
 3   Monthly_Inhand_Salary     84998 non-null   float64
 4   Num_Bank_Accounts         100000 non-null  int64  
 5   Num_Credit_Card           100000 non-null  int64  
 6   Interest_Rate             100000 non-null  int64  
 7   Num_of_Loan               100000 non-null  object 
 8   Type_of_Loan              88592 non-null   object 
 9   Delay_from_due_date       100000 non-null  int64  
 10  Num_of_Delayed_Payment    92998 non-null   object 
 11  Changed_Credit_Limit      100000 non-null  object 
 12  Num_Credit_Inquiries      98035 non-null   float64
 13  Credit_Mix                100000 non-null  ob

In [30]:
data["Age"] = pd.to_numeric(data["Age"], errors="coerce")
data["Annual_Income"] = pd.to_numeric(data["Annual_Income"], errors="coerce")
data["Monthly_Inhand_Salary"] = pd.to_numeric(
    data["Monthly_Inhand_Salary"], errors="coerce"
)
data["Num_Bank_Accounts"] = pd.to_numeric(data["Num_Bank_Accounts"], errors="coerce")
data["Num_Credit_Card"] = pd.to_numeric(data["Num_Credit_Card"], errors="coerce")
data["Interest_Rate"] = pd.to_numeric(data["Interest_Rate"], errors="coerce")
data["Num_of_Loan"] = pd.to_numeric(data["Num_of_Loan"], errors="coerce")
data["Delay_from_due_date"] = pd.to_numeric(
    data["Delay_from_due_date"], errors="coerce"
)
data["Num_of_Delayed_Payment"] = pd.to_numeric(
    data["Num_of_Delayed_Payment"], errors="coerce"
)
data["Changed_Credit_Limit"] = pd.to_numeric(
    data["Changed_Credit_Limit"], errors="coerce"
)
data["Num_Credit_Inquiries"] = pd.to_numeric(
    data["Num_Credit_Inquiries"], errors="coerce"
)
data["Outstanding_Debt"] = pd.to_numeric(data["Outstanding_Debt"], errors="coerce")
data["Credit_Utilization_Ratio"] = pd.to_numeric(
    data["Credit_Utilization_Ratio"], errors="coerce"
)
data["Total_EMI_per_month"] = pd.to_numeric(
    data["Total_EMI_per_month"], errors="coerce"
)

data["Amount_invested_monthly"] = pd.to_numeric(
    data["Amount_invested_monthly"], errors="coerce"
)
data["Monthly_Balance"] = pd.to_numeric(
    data["Monthly_Balance"], errors="coerce"
)



data.Age.fillna(data.Age.mean(), inplace=True)
data.Annual_Income.fillna(data.Annual_Income.mean(), inplace=True)
data.Monthly_Inhand_Salary.fillna(data.Monthly_Inhand_Salary.mean(), inplace=True)
data.Num_of_Loan.fillna(data.Num_of_Loan.mean(), inplace=True)
data.Type_of_Loan.fillna(method='ffill', inplace=True)
data.Num_of_Delayed_Payment.fillna(method='ffill', inplace=True)
data.Changed_Credit_Limit.fillna(method='ffill', inplace=True)
data.Num_Credit_Inquiries.fillna(data.Num_Credit_Inquiries.mean(), inplace=True)
data.Outstanding_Debt.fillna(data.Outstanding_Debt.mean(), inplace=True)
data.Credit_History_Age.fillna(method="ffill", inplace=True)
data.Amount_invested_monthly.fillna(method="ffill", inplace=True)
data.Monthly_Balance.fillna(data.Monthly_Balance.mean(), inplace=True)

print("occupation_le")
occupation_le = LabelEncoder()
data.Occupation = occupation_le.fit_transform(data.Occupation)
print("type_of_loan_le")
type_of_loan_le = LabelEncoder()
data.Type_of_Loan = type_of_loan_le.fit_transform(data.Type_of_Loan)
print("credit_mix_le")
credit_mix_le = LabelEncoder()
data.Credit_Mix = credit_mix_le.fit_transform(data.Credit_Mix)
print("credit_history_age_le")
credit_history_age_le = LabelEncoder()
data.Credit_History_Age = credit_history_age_le.fit_transform(data.Credit_History_Age)
print("payment_of_min_amount_le")
payment_of_min_amount_le = LabelEncoder()
data.Payment_of_Min_Amount = payment_of_min_amount_le.fit_transform(data.Payment_of_Min_Amount)
print("payment_behaviour_le")
payment_behaviour_le = LabelEncoder()
data.Payment_Behaviour = payment_behaviour_le.fit_transform(data.Payment_Behaviour)
print("credit_score_le")
credit_score_le = LabelEncoder()
data.Credit_Score = credit_score_le.fit_transform(data.Credit_Score)

data.info()


occupation_le
type_of_loan_le
credit_mix_le
credit_history_age_le
payment_of_min_amount_le
payment_behaviour_le
credit_score_le
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 23 columns):
 #   Column                    Non-Null Count   Dtype  
---  ------                    --------------   -----  
 0   Age                       100000 non-null  float64
 1   Occupation                100000 non-null  int64  
 2   Annual_Income             100000 non-null  float64
 3   Monthly_Inhand_Salary     100000 non-null  float64
 4   Num_Bank_Accounts         100000 non-null  int64  
 5   Num_Credit_Card           100000 non-null  int64  
 6   Interest_Rate             100000 non-null  int64  
 7   Num_of_Loan               100000 non-null  float64
 8   Type_of_Loan              100000 non-null  int64  
 9   Delay_from_due_date       100000 non-null  int64  
 10  Num_of_Delayed_Payment    100000 non-null  float64
 11  Changed_Credit_Limit      100

In [ ]:
data.head(n=20)

## Modeling

In [31]:
# data = data.drop(
#     [
# "Occupation",
# "Type_of_Loan",
# "Delay_from_due_date",
# "Num_of_Delayed_Payment",
# "Changed_Credit_Limit",
# "Payment_of_Min_Amount",
# "Payment_Behaviour",
#     ],
#     axis=1,
# )


x_train, x_test, y_train, y_test = train_test_split(
    data.drop("Credit_Score", axis=1), data.Credit_Score, test_size=0.15, random_state=42
)
scaler = StandardScaler()
x_train = scaler.fit_transform(x_train)
x_test = scaler.transform(x_test)


In [32]:
rf = RandomForestClassifier(n_estimators=5000, max_depth=200, random_state=2023, n_jobs=-1, verbose=1)
rf.fit(x_train, y_train)
y_pred = rf.predict(x_test)
print("Accuracy: ", accuracy_score(y_test, y_pred))


[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:    1.4s
[Parallel(n_jobs=-1)]: Done 184 tasks      | elapsed:    7.0s
[Parallel(n_jobs=-1)]: Done 434 tasks      | elapsed:   15.7s
[Parallel(n_jobs=-1)]: Done 784 tasks      | elapsed:   27.7s
[Parallel(n_jobs=-1)]: Done 1234 tasks      | elapsed:   43.9s
[Parallel(n_jobs=-1)]: Done 1784 tasks      | elapsed:  1.1min
[Parallel(n_jobs=-1)]: Done 2434 tasks      | elapsed:  1.5min
[Parallel(n_jobs=-1)]: Done 3184 tasks      | elapsed:  2.0min
[Parallel(n_jobs=-1)]: Done 4034 tasks      | elapsed:  2.5min
[Parallel(n_jobs=-1)]: Done 4984 tasks      | elapsed:  3.2min
[Parallel(n_jobs=-1)]: Done 5000 out of 5000 | elapsed:  3.2min finished
[Parallel(n_jobs=8)]: Using backend ThreadingBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done  34 tasks      | elapsed:    0.0s
[Parallel(n_jobs=8)]: Done 184 tasks      | elapsed:    0.1s
[Parallel(n_jobs=8

Accuracy:  0.7926


[Parallel(n_jobs=8)]: Done 4984 tasks      | elapsed:    4.4s
[Parallel(n_jobs=8)]: Done 5000 out of 5000 | elapsed:    4.4s finished


In [33]:
# Use knn to predict the credit score
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors=4, n_jobs=-1)
knn.fit(x_train, y_train)
y_pred = knn.predict(x_test)
print("Accuracy: ", accuracy_score(y_test, y_pred))
print("Confusion Matrix: ", confusion_matrix(y_test, y_pred))


Accuracy:  0.6275333333333334
Confusion Matrix:  [[1514  209  904]
 [ 386 2853 1182]
 [1262 1644 5046]]
